<a href="https://colab.research.google.com/github/ShaliniR8/LPAQA/blob/master/Copy_of_LAMA_Dataset_Stuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/tuner007/pegasus_paraphrase   
https://github.com/jzbjyb/LPAQA  
https://huggingface.co/datasets/lama  



In [ ]:
%%capture
!pip install transformers
!pip install datasets

In [ ]:
%%capture
!pip install sentencepiece

LPAQA train - https://github.com/ShaliniR8/LPAQA/blob/master/TREx_train.zip

In [ ]:
%%capture
!pip install jsonlines
!unzip 'TREx_train.zip' -d dataset

In [ ]:
# Troubleshooting: https://stackoverflow.com/questions/65854722/huggingface-albert-tokenizer-nonetype-error-with-colab

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase' 
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

# paraphrase method
# return string[]? - list of paraphrases of input_text
def paraphrase_pegasus(input_text, num_return_sequences, num_beams=10):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset("lama")
dataset

No config specified, defaulting to: lama/trex


Generating train split:   0%|          | 0/1304391 [00:00<?, ? examples/s]

Dataset lama downloaded and prepared to /root/.cache/huggingface/datasets/lama/trex/1.1.0/430016dd70224564ad385a96e0e4a3f88aeb5beaf4e34a8cf65b390fbc83aed7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['uuid', 'obj_uri', 'obj_label', 'sub_uri', 'sub_label', 'predicate_id', 'sub_surface', 'obj_surface', 'masked_sentence', 'template', 'template_negated', 'label', 'description', 'type'],
        num_rows: 1304391
    })
})

In [ ]:
# go from knowledge triples to prompts? 

# 41 of these manually written partial-prompts:


# “x shares a border with y”
# "France shares a border with Germany"
# paraphrase()
# "France borders Germany"
# delete 'France' and 'Germany'
# "x borders y"


#LAMA dataset - x shares a border with y -----> transformer -> [ x borders y, y is a border of x, .... ]



# paraphrase what??? paraphrase(“France shares a border with Germany”)
# paraphrase(“x shares a border with y”, x='France')

In [ ]:
# dataset["train"].shape

In [ ]:
# def fun(examples):
#   dic = examples[:50]
#   sub_labels = dic["sub_label"]
#   for label in sub_labels:
#     # print(label)
#     if len(label.split()) == 1: print(label)

# fun(dataset["train"])

In [ ]:
# dataset["train"][5]

In [ ]:
train_df = dataset["train"].to_pandas()
train_df.sample(5)

,uuid,obj_uri,obj_label,sub_uri,sub_label,predicate_id,sub_surface,obj_surface,masked_sentence,template,template_negated,label,description,type
854060,e41a2c44-76f4-4c15-8212-d1c357edec4a,Q29,Spain,Q38,Italy,P530,Italy,Spain,"The series is published by Glénat in [MASK], b...",[X] maintains diplomatic relations with [Y] .,[X] does not maintain diplomatic relations wit...,diplomatic relation,diplomatic relations of the country,N-M
1060629,b70505ef-bae4-46e8-a512-0b5e0b6d658b,Q17,Japan,Q30,United States of America,P530,American,Japan,"In baseball, Yonamine was the first American t...",[X] maintains diplomatic relations with [Y] .,[X] does not maintain diplomatic relations wit...,diplomatic relation,diplomatic relations of the country,N-M
1045870,a8df3dcb-d16d-46c1-9f81-419cea776c9c,Q38,Italy,Q142,France,P530,France,Italy,"This beetle is distributed in Belarus, Finland...",[X] maintains diplomatic relations with [Y] .,[X] does not maintain diplomatic relations wit...,diplomatic relation,diplomatic relations of the country,N-M
1259837,0a5208ee-fbcb-4563-971a-3097e799a8db,Q10690,Cardiff,Q25,Wales,P36,Wales,Cardiff,Pwllfanogl is a hamlet in the community of Lla...,The capital of [X] is [Y] .,The capital of [X] is not [Y] .,capital,"primary city of a country, state or other type...",1-1
1025365,b926dcfc-2947-4dbf-b3af-2c733e336c52,Q16,Canada,Q27,Ireland,P530,Ireland,Canada,"He was born at Castle Waller in Newport, Count...",[X] maintains diplomatic relations with [Y] .,[X] does not maintain diplomatic relations wit...,diplomatic relation,diplomatic relations of the country,N-M


In [ ]:
# GOAL: get a list of prompt templates for each relation
# for each each relation 
  # loop over every training example
  # plug subject and object into template
  # new_sentences = paraphrase
  # for st in new_sentences:
    # if subject and object (strings) in st:
      # replace subject and object with x and y
      # add to final list? set?
import pandas as pd


subset_dict = {}

# get list of relations
relations = train_df['predicate_id'].unique()

# given a row, add generated prompts to a global list
def generate_prompts(row):
  template = row['template']
  # orig_prompt = template.replace('[X]', row['sub_label']).replace('[Y]', row['obj_label'])   # plug subject and object into template
  
  # feed template into paraphraser as is (but remove brackets first)
  orig_prompt = template.replace("]", "").replace("[", "") 
  new_sentences = paraphrase_pegasus(orig_prompt, 40, num_beams=40) # list?
  new_sentences = [sentence.replace(' X ', ' [X] ').replace(' Y ', ' [Y] ').replace(' X.', ' [X].').replace(' Y.', ' [Y].').replace('X ', '[X] ').replace('Y ', '[Y] ') for sentence in new_sentences]
  # TODO: check no multiple masks !!!!!!!!!!!!!!!
  print(new_sentences)
  return new_sentences

for r in relations:
  subset_df = train_df[train_df['predicate_id'] == r].sample(1)
  subset_df['generated_prompts'] = subset_df.apply(generate_prompts, axis=1)
  subset_dict[r] = subset_df


['[Y] is where [X] is located.', '[Y] is the location of [X].', 'The location of [X] is in [Y].', '[X] is in [Y].', 'There is a place called [X] located in [Y].', 'The location of [X] is [Y].', 'There is a place called [X] in [Y].', '[Y] is located in [X].', 'You can find [X] in [Y].', '[X] is located in [Y].', 'In [Y] is where [X] is located.', '[Y] is the location for [X].', '[Y] is the place where [X] is located.', '[X] is in the same location as [Y].', 'In Y, there is [X].', 'The place called [X] is located in [Y].', '[Y] is where [X] is.', '[X] is in the same area as [Y].', '[X] is in the same place as [Y].', 'There is [X] in [Y].', 'There is [X] located in [Y].', 'It is located in [Y].', 'In [Y] is the location of [X].', 'There is a street called [X] located in [Y].', '[Y] has [X] located in it.', 'There is a place named [X] located in [Y].', 'There is a street called [X] in [Y].', 'There is a person located in [Y].', '[X] is in the same city as [Y].', '[X] can be found in [Y].',

In [ ]:
# NEXT STEPS
# rank the prompts based on training accuracy?
# report top-5 ensemble performance

In [ ]:
import jsonlines
import os

def get_pairs(r):
  # checking TREX_TRAIN
  # print(f"For relation {r}\n")
  jsonl = "dataset/TREx_train"
  pairs = []
  with jsonlines.open(os.path.join(jsonl, r+".jsonl")) as reader:
    for obj in reader: 
      # print(obj)
      pairs.append([obj["sub_label"], obj["obj_label"]]) # this is all
      # return [obj["sub_label"], obj["obj_label"]]
  return pairs[:10]

In [ ]:
# print(pairs) for a relation
# [['Kjerkhågen', 'Nordland'], ['Dadow', 'Mecklenburg-Vorpommern'], ['Bergan', 'Nordland'], ['Bystřinova', 'Olomouc'], ['Schaepmanlaan', 'Kampen'],...

In [ ]:
# replace subjects with training subject and object with [MASK]

def probe_prompts(new_subset):
  generated_prompts = new_subset["generated_prompts"][0]
  probed_prompts = []
  sub_label = new_subset["pairs"][0][0]
  for prompt in generated_prompts:
    prompt = prompt.replace("[X]", sub_label).replace("[Y]", "[MASK]")
    probed_prompts.append(prompt)
  # print(probed_prompts)
  return [probed_prompts]


# for r in relations:
#   subset_df = subset_dict[r]
#   subset_df['pairs'] = [get_pairs(r)]
#   subset_ = subset_df


new_subset_dict = {}
for key, df in subset_dict.items(): 
    pairs = get_pairs(key)
    for pair in pairs:
      new_subset = pd.DataFrame({"generated_prompts": list(df['generated_prompts']), "pairs": [pair]})
      # print("New dataframe: ", new_subset["pairs"])
      new_subset['probed_prompts'] = probe_prompts(new_subset)
      new_subset_dict[key] = new_subset
print("New dictionary of datasets:", new_subset_dict["P937"]["pairs"])


New dictionary of datasets: 0    [Exekias, Attica]
Name: pairs, dtype: object


In [ ]:
def accuracy(dic): # predictions --> list, actual -> word
  def evaluate(r, dic):
    actual = dic[r]["actual"]
    predictions = dic[r]["predictions"]
    if actual.lower() in predictions: return 1
    else: return 0

  Sum = 0
  relations = dic.keys()
  for r in relations:
    Sum += evaluate(r, dic)

  micro = Sum/len(relations)
  return micro



# MASKED MODEL

In [ ]:
from transformers import pipeline

In [ ]:
unmasker = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def filter(sentences):
  final = []
  for sentence in sentences:
    if "[MASK]" in sentence:
      final.append(sentence)
  return final


In [ ]:
def predict(prompts, unmasker):
  predicted = []
  for prompt in prompts:
    # print(prompt)
    try:
      p = unmasker(prompt)[0]["token_str"]
      predicted.append(p)
    except: pass
  return predicted


In [ ]:
relation_predictions = {}
for key, df in new_subset_dict.items():
  actual = df["pairs"][0][1]
  filtered_prompts = filter(df["probed_prompts"][0])
  predicted = predict(filtered_prompts, unmasker)
  relation_predictions[key] = [predicted, actual]

# Predictions

In [ ]:
dic = {}
for key, value in relation_predictions.items():
  dic[key] = {"predictions": value[0], "actual": value[1]}

In [ ]:
accuracy(dic)

0.17073170731707318